### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url_nyt = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)

In [4]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19

for page in range(20):
    
    
    # create query with a page number
    query_url = query_url_nyt + '&page=' + str(page)
    
    # API results show 10 articles at a time
    # Make a "GET" request and retrieve the JSON
    try:
        response = requests.get(query_url)
        response.raise_for_status()  # Raises an HTTPError for bad responses

        # Check if there are any documents to process
        if response.json()['response']['docs']:
            # Loop through the 'docs' and append each review to the list
            for doc in response.json()['response']['docs']:
                reviews_list.append(doc)
            print(f"Page {page} retrieved and processed.")
        else:
            # Print the page number with no results and break from the loop
            print(f"No results on page {page}, stopping.")
            break

    except requests.RequestException as e:
        print(f"Request failed: {e}")
        break

    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    

Page 0 retrieved and processed.
Page 1 retrieved and processed.
Page 2 retrieved and processed.
Page 3 retrieved and processed.
Page 4 retrieved and processed.
Page 5 retrieved and processed.
Page 6 retrieved and processed.
Page 7 retrieved and processed.
Page 8 retrieved and processed.
Page 9 retrieved and processed.
Page 10 retrieved and processed.
Page 11 retrieved and processed.
Page 12 retrieved and processed.
Page 13 retrieved and processed.
Page 14 retrieved and processed.
Page 15 retrieved and processed.
Page 16 retrieved and processed.
Page 17 retrieved and processed.
Page 18 retrieved and processed.
Page 19 retrieved and processed.


In [5]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
if len(reviews_list) >= 5:
    # Print the first 5 reviews in a formatted JSON style
    print(json.dumps(reviews_list[:5], indent=4))
else:
    # If there are fewer than 5 reviews, print whatever is available
    print(json.dumps(reviews_list, indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [13]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(
    reviews_list, 
    meta=[
        'web_url',
        'snippet',
        'source',
        ['headline', 'main'],  # Access nested data under 'headline'
        ['headline', 'print_headline'],
        'pub_date',
        ['byline', 'original'],
        'word_count'
    ],
    errors='ignore'  # To ignore missing data in any of the paths
)

# Display the first few rows of the DataFrame to check the output
print(reviews_df.head())

                                             web_url  \
0  https://www.nytimes.com/2023/05/25/movies/the-...   
1  https://www.nytimes.com/2023/05/04/movies/what...   
2  https://www.nytimes.com/2023/05/04/movies/you-...   
3  https://www.nytimes.com/2023/04/21/movies/a-to...   
4  https://www.nytimes.com/2023/04/20/movies/othe...   

                                             snippet              source  \
0  A gynecologist and her patient form a horrifyi...  The New York Times   
1  Two childhood friends navigate cultural differ...  The New York Times   
2  Religion comes between two girls falling in lo...  The New York Times   
3  Rachael Leigh Cook stars in this bland rom-com...  The New York Times   
4  A radiant Virginie Efira stars as a Parisian t...  The New York Times   

                                            keywords  \
0  [{'name': 'subject', 'value': 'Movies', 'rank'...   
1  [{'name': 'subject', 'value': 'Movies', 'rank'...   
2  [{'name': 'subject', 'value': 'Movi

In [14]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
# Regular expression to extract text between ‘ and ’ followed by " Review"
pattern = r"‘(.+?)’ Review"

# Extracting the title and creating a new column
reviews_df['title'] = reviews_df['headline.main'].str.extract(pattern, expand=True)



In [15]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
# Function to fix the "keywords" column by converting cells from a list to a string
def fix_keywords_column(df):
    # Apply the extract_keywords function to each cell in the 'keywords' column
    df['keywords'] = df['keywords'].apply(extract_keywords)
    return df
# Call new function:
reviews_df = fix_keywords_column(reviews_df)



In [18]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
reviews_df['title'].tolist()

['The Attachment Diaries',
 nan,
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Cloud',
 'A Journal for Jordan',
 'West Side St

### Access The Movie Database API

In [17]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [ ]:
# Create an empty list to store the results
reviews_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests


# Loop through the titles

    # Check if we need to sleep before making a request


    # Add 1 to the request counter

    
    # Perform a "GET" request for The Movie Database


    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.

        # Get movie id


        # Make a request for a the full movie details


        # Execute "GET" request with url

        
        # Extract the genre names into a list


        # Extract the spoken_languages' English name into a list


        # Extract the production_countries' name into a list


        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list

        
        # Print out the title that was found



In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data


In [ ]:
# Convert the results to a DataFrame


### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title


In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


In [ ]:
# Drop "byline.person" column


In [ ]:
# Delete duplicate rows and reset index


In [ ]:
# Export data to CSV without the index
